## Ampatin CNN Exercise

In [1]:
#import libraries needed
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import numpy as np
import cv2
import os

#import tools needed
from keras.backend import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import Sequential

In [2]:
#getting and training the data
train_data = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

train_path = 'C:/Users/camil/Downloads/Covid19/train'
train_dataset = train_data.flow_from_directory(train_path, 
                                          target_size = (224, 224), 
                                          batch_size = 32, 
                                          class_mode = 'categorical')

Found 249 images belonging to 3 classes.


In [3]:
#getting and testing the data
test_data = ImageDataGenerator(rescale=1./255)
test_path = 'C:/Users/camil/Downloads/Covid19/test'
test_dataset = test_data.flow_from_directory(test_path, 
                                         target_size = (224, 224), 
                                         batch_size = 32, 
                                         class_mode = 'categorical')

Found 65 images belonging to 3 classes.


In [4]:
#modelling the data
model = Sequential()
model.add(Conv2D(32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=(224,224,3)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 110, 110, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                    

In [5]:
#fitting the data
history = model.fit_generator(train_dataset,steps_per_epoch=7,epochs = 5,validation_data = test_dataset,validation_steps=2)

C:\Users\camil\AppData\Local\Temp\ipykernel_1744\4011286485.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_dataset,steps_per_epoch=7,epochs = 5,validation_data = test_dataset,validation_steps=2)


Epoch 1/5
7/7 [==============================] - 152s 21s/step - loss: 1.7061 - accuracy: 0.3733 - val_loss: 1.0744 - val_accuracy: 0.3906
Epoch 2/5
7/7 [==============================] - 143s 20s/step - loss: 0.9630 - accuracy: 0.5530 - val_loss: 0.9905 - val_accuracy: 0.5781
Epoch 3/5
7/7 [==============================] - 153s 21s/step - loss: 0.7816 - accuracy: 0.6820 - val_loss: 0.8137 - val_accuracy: 0.6562
Epoch 4/5
7/7 [==============================] - 151s 22s/step - loss: 0.6487 - accuracy: 0.7327 - val_loss: 0.6570 - val_accuracy: 0.7656
Epoch 5/5
7/7 [==============================] - 130s 19s/step - loss: 0.5826 - accuracy: 0.7788 - val_loss: 0.6741 - val_accuracy: 0.6719


In [6]:
#getting the accuracy
accuracy = model.evaluate(train_dataset)
print('Test accuracy:', accuracy[1])

8/8 [==============================] - 47s 6s/step - loss: 0.5347 - accuracy: 0.8353
Test accuracy: 0.8353413939476013


In [8]:
#using the model and predicting for the validation
dataset = 'C:/Users/camil/Downloads/Covid19/'

def loadImages(path, folder):
    data_files = sorted([os.path.join(path, folder, file)
                        for file in os.listdir(path + folder + '/')
                        if (file.endswith('.jpg') or file.endswith('.jpeg'))])
    return data_files

validation = loadImages(dataset, 'validation')

g = validation
f = []
images = []

for x in g:
    img = cv2.imread(x)
    img = img[:, :, :3]
    images.append(img)

    width = 224
    height = 224
    dim = (width, height)

    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    f.append(resized)

f = np.array(f)
f = f/255

predict = model.predict(f)
predictions = np.argmax(predict, axis=1)
print(predictions)

1/1 [==============================] - 1s 1s/step
[1 1 0 2 2 2 0 0 2 0]
